In [ ]:
import os
import math
import string
from PIL import Image, ImageDraw, ImageFont
from google.colab import drive
from google.colab import files

# 1. Montar o Google Drive
drive.mount('/content/drive', force_remount=True)

def criar_colagem_vertical_letras_centro(diretorio_alvo, nome_saida='colagem_vertical_letras.jpg'):
    """
    Cria uma colagem vertical de 01 a 24.
    Se a imagem não existir, plota a LETRA DA SEQUÊNCIA (A, B, C...)
    centralizada no espaço em branco.
    """

    diretorio_alvo = diretorio_alvo.strip()
    if not os.path.exists(diretorio_alvo):
        print(f"❌ ERRO: Diretório não encontrado: {diretorio_alvo}")
        return

    print("Iniciando varredura de slots (01 a 24)...")

    extensoes = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif']
    imagens_slots = []

    # Variáveis para dimensão máxima
    max_w = 0
    max_h = 0
    imagens_encontradas = 0

    # 2. Carregar imagens ou marcar ausência
    for i in range(1, 25): # 1 a 24
        nome_base = f"{i:02d}" # "01", "02"...
        img_encontrada = None

        for ext in extensoes:
            caminho_teste = os.path.join(diretorio_alvo, nome_base + ext)
            caminho_teste_upper = os.path.join(diretorio_alvo, nome_base + ext.upper())

            if os.path.exists(caminho_teste):
                img_encontrada = caminho_teste
                break
            elif os.path.exists(caminho_teste_upper):
                img_encontrada = caminho_teste_upper
                break

        if img_encontrada:
            try:
                img = Image.open(img_encontrada).convert('RGB')
                imagens_slots.append(img)
                if img.width > max_w: max_w = img.width
                if img.height > max_h: max_h = img.height
                imagens_encontradas += 1
            except:
                imagens_slots.append(None)
        else:
            imagens_slots.append(None) # Slot vazio

    # Se não achou nenhuma imagem para basear o tamanho, cria um padrão
    if max_w == 0: max_w = 1000
    if max_h == 0: max_h = 1000

    # 3. Configurar Canvas
    largura_celula = max_w
    altura_celula = max_h
    num_slots = 24

    largura_total = largura_celula
    altura_total = num_slots * altura_celula

    print(f"Gerando Canvas: {largura_total}x{altura_total} pixels.")
    canvas = Image.new('RGB', (largura_total, altura_total), (255, 255, 255))

    # Carregar fontes
    # Fonte PEQUENA para o canto (quando tem imagem)
    tamanho_fonte_canto = int(altura_celula * 0.08)

    # Fonte GRANDE para o centro (quando não tem imagem)
    tamanho_fonte_centro = int(altura_celula * 0.50) # 50% da altura da célula

    try:
        fonte_canto = ImageFont.truetype("LiberationSans-Regular.ttf", tamanho_fonte_canto)
        fonte_centro = ImageFont.truetype("LiberationSans-Regular.ttf", tamanho_fonte_centro)
    except:
        fonte_canto = ImageFont.load_default()
        fonte_centro = ImageFont.load_default() # O default não escala bem, mas evita crash

    alfabeto = string.ascii_uppercase
    draw = ImageDraw.Draw(canvas)

    # 4. Montagem
    for i in range(num_slots):
        img = imagens_slots[i]
        x = 0
        y = i * altura_celula

        # Define qual é a letra: 0->A, 1->B ... 23->X ...
        letra = alfabeto[i] if i < 26 else f"Z{i}"

        if img:
            # --- CASO 1: IMAGEM EXISTE ---
            # Redimensiona e cola a imagem
            img_final = img.resize((largura_celula, altura_celula), Image.LANCZOS)
            canvas.paste(img_final, (x, y))

            # Desenha a letra pequena no canto (padrão científico)
            margem = int(largura_celula * 0.03)
            pos_txt = (x + margem, y + margem)

            bbox = draw.textbbox(pos_txt, letra, font=fonte_canto)
            draw.rectangle((bbox[0]-10, bbox[1]-10, bbox[2]+10, bbox[3]+10), fill="black")
            draw.text(pos_txt, letra, fill="white", font=fonte_canto)

        else:
            # --- CASO 2: IMAGEM FALTANDO ---
            # O fundo já é branco. Vamos desenhar a letra GRANDE no centro.

            # Usamos textbbox para calcular o centro exato da letra
            bbox_centro = draw.textbbox((0, 0), letra, font=fonte_centro)
            largura_texto = bbox_centro[2] - bbox_centro[0]
            altura_texto = bbox_centro[3] - bbox_centro[1]

            cx = x + (largura_celula - largura_texto) / 2
            cy = y + (altura_celula - altura_texto) / 2

            # Desenha a letra em PRETO (ou cinza escuro) no meio do espaço branco
            draw.text((cx, cy - (altura_texto * 0.2)), letra, fill="black", font=fonte_centro)

    # 5. Salvar
    caminho_final = os.path.join(diretorio_alvo, nome_saida)
    Image.MAX_IMAGE_PIXELS = None

    canvas.save(caminho_final, format='JPEG', quality=100, subsampling=0, dpi=(700, 700))

    print(f"✅ Arquivo salvo: {caminho_final}")
    files.download(caminho_final)

# --- EXECUÇÃO ---
pasta_alvo = '/content/drive/MyDrive/pesquisa_mestrado_2025/unir_fotos/quarto_topico_figura 1'
criar_colagem_vertical_letras_centro(pasta_alvo)

Mounted at /content/drive
Iniciando varredura de slots (01 a 24)...
Gerando Canvas: 1563x28008 pixels.
✅ Arquivo salvo: /content/drive/MyDrive/pesquisa_mestrado_2025/unir_fotos/quarto_topico_figura 1/colagem_vertical_letras.jpg


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
import os
import math
import string
from PIL import Image, ImageDraw, ImageFont
from google.colab import drive
from google.colab import files

# 1. Montar o Google Drive
drive.mount('/content/drive', force_remount=True)

def criar_colagem_grid_4_colunas(diretorio_alvo, nome_saida='colagem_grid_4cols.jpg'):
    """
    Cria uma colagem de 24 imagens dispostas em 4 COLUNAS (grade 4x6).
    Se a imagem não existir, plota a LETRA DA SEQUÊNCIA (A, B, C...)
    centralizada no espaço em branco.
    """

    diretorio_alvo = diretorio_alvo.strip()
    if not os.path.exists(diretorio_alvo):
        print(f"❌ ERRO: Diretório não encontrado: {diretorio_alvo}")
        return

    print("Iniciando varredura de slots (01 a 24)...")

    extensoes = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif']
    imagens_slots = []

    # Variáveis para dimensão máxima
    max_w = 0
    max_h = 0
    imagens_encontradas = 0

    # 2. Carregar imagens ou marcar ausência
    for i in range(1, 25): # 1 a 24
        nome_base = f"{i:02d}" # "01", "02"...
        img_encontrada = None

        for ext in extensoes:
            caminho_teste = os.path.join(diretorio_alvo, nome_base + ext)
            caminho_teste_upper = os.path.join(diretorio_alvo, nome_base + ext.upper())

            if os.path.exists(caminho_teste):
                img_encontrada = caminho_teste
                break
            elif os.path.exists(caminho_teste_upper):
                img_encontrada = caminho_teste_upper
                break

        if img_encontrada:
            try:
                img = Image.open(img_encontrada).convert('RGB')
                imagens_slots.append(img)
                if img.width > max_w: max_w = img.width
                if img.height > max_h: max_h = img.height
                imagens_encontradas += 1
            except:
                imagens_slots.append(None)
        else:
            imagens_slots.append(None) # Slot vazio

    # Se não achou nenhuma imagem para basear o tamanho, cria um padrão
    if max_w == 0: max_w = 1000
    if max_h == 0: max_h = 1000

    # 3. Configurar Canvas (MODIFICADO PARA 4 COLUNAS)
    largura_celula = max_w
    altura_celula = max_h
    num_slots = 24

    colunas = 6
    linhas = math.ceil(num_slots / colunas) # 24 / 4 = 6 linhas

    largura_total = colunas * largura_celula
    altura_total = linhas * altura_celula

    print(f"Gerando Canvas: {largura_total}x{altura_total} pixels ({colunas} colunas x {linhas} linhas).")
    canvas = Image.new('RGB', (largura_total, altura_total), (255, 255, 255))

    # Carregar fontes
    # Fonte PEQUENA para o canto (quando tem imagem)
    tamanho_fonte_canto = int(altura_celula * 0.08)

    # Fonte GRANDE para o centro (quando não tem imagem)
    tamanho_fonte_centro = int(altura_celula * 0.50)

    try:
        fonte_canto = ImageFont.truetype("LiberationSans-Regular.ttf", tamanho_fonte_canto)
        fonte_centro = ImageFont.truetype("LiberationSans-Regular.ttf", tamanho_fonte_centro)
    except:
        fonte_canto = ImageFont.load_default()
        fonte_centro = ImageFont.load_default()

    alfabeto = string.ascii_uppercase
    draw = ImageDraw.Draw(canvas)

    # 4. Montagem
    for i in range(num_slots):
        img = imagens_slots[i]

        # --- LÓGICA DE GRADE (GRID) ---
        coluna_atual = i % colunas       # 0, 1, 2, 3
        linha_atual = i // colunas       # 0, 1, 2, 3, 4, 5

        x = coluna_atual * largura_celula
        y = linha_atual * altura_celula

        # Define qual é a letra: 0->A, 1->B ...
        letra = alfabeto[i] if i < 26 else f"Z{i}"

        if img:
            # --- CASO 1: IMAGEM EXISTE ---
            img_final = img.resize((largura_celula, altura_celula), Image.LANCZOS)
            canvas.paste(img_final, (x, y))

            # Desenha a letra pequena no canto
            margem = int(largura_celula * 0.03)
            pos_txt = (x + margem, y + margem)

            bbox = draw.textbbox(pos_txt, letra, font=fonte_canto)
            draw.rectangle((bbox[0]-10, bbox[1]-10, bbox[2]+10, bbox[3]+10), fill="black")
            draw.text(pos_txt, letra, fill="white", font=fonte_canto)

        else:
            # --- CASO 2: IMAGEM FALTANDO ---
            # Centralizar letra no espaço em branco
            bbox_centro = draw.textbbox((0, 0), letra, font=fonte_centro)
            largura_texto = bbox_centro[2] - bbox_centro[0]
            altura_texto = bbox_centro[3] - bbox_centro[1]

            cx = x + (largura_celula - largura_texto) / 2
            cy = y + (altura_celula - altura_texto) / 2

            draw.text((cx, cy - (altura_texto * 0.2)), letra, fill="black", font=fonte_centro)

    # 5. Salvar
    caminho_final = os.path.join(diretorio_alvo, nome_saida)
    Image.MAX_IMAGE_PIXELS = None

    # Salvando com alta qualidade
    canvas.save(caminho_final, format='JPEG', quality=100, subsampling=0, dpi=(700, 700))

    print(f"✅ Arquivo salvo: {caminho_final}")
    files.download(caminho_final)

# --- EXECUÇÃO ---
# Atualize o caminho se necessário
pasta_alvo = '/content/drive/MyDrive/pesquisa_mestrado_2025/unir_fotos/quarto_topico_figura 1'
criar_colagem_grid_4_colunas(pasta_alvo)

Mounted at /content/drive
Iniciando varredura de slots (01 a 24)...
Gerando Canvas: 9408x6272 pixels (6 colunas x 4 linhas).
✅ Arquivo salvo: /content/drive/MyDrive/pesquisa_mestrado_2025/unir_fotos/quarto_topico_figura 1/colagem_grid_4cols.jpg


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>